In [1]:
import util
import plots
import numpy as np
import plan_params as pp
import pandas as pd
import inputs_for_mv
import ts_inputs as ts
import plotly.graph_objects as go
import plotly.express as px
import os

In [2]:
output_df = ts.get_summarized_output('return_data.xlsx')
output_df

,FS AdjWeights,Return,Vol,Liability,15+ STRIPS,Long Corporate,Ultra 30-Year UST Futures,Equity,Liquid Alternatives,Private Equity,Credit,Real Estate,Equity Hedges,Cash
Liability,-1.0000,0.092548,0.121462,1.000000,0.785004,0.900257,0.775819,0.045897,0.354423,-0.026009,0.212992,0.148620,-0.179998,-0.020247
15+ STRIPS,0.1428,0.090032,0.174249,0.785004,1.000000,0.596345,0.993248,-0.400766,0.254151,0.051969,-0.294603,0.035470,0.179479,-0.064554
Long Corporate,0.1428,0.075284,0.089549,0.900257,0.596345,1.000000,0.593998,0.316945,0.360452,-0.045083,0.500296,0.354230,-0.376090,0.015852
Ultra 30-Year UST Futures,0.0000,0.053656,0.120932,0.775819,0.993248,0.593998,1.000000,-0.395814,0.246743,0.057720,-0.303945,0.043955,0.201600,-0.067388
Equity,0.4386,0.104995,0.127950,0.045897,-0.400766,0.316945,-0.395814,1.000000,0.291249,-0.021922,0.803769,0.432745,-0.476481,0.020896
Liquid Alternatives,0.0918,0.010168,0.040302,0.354423,0.254151,0.360452,0.246743,0.291249,1.000000,0.062601,0.151844,0.087186,-0.069778,-0.004997
Private Equity,0.0816,0.145317,0.047217,-0.026009,0.051969,-0.045083,0.057720,-0.021922,0.062601,1.000000,-0.158051,0.174372,0.033113,-0.190188
Credit,0.0510,0.057278,0.058175,0.212992,-0.294603,0.500296,-0.303945,0.803769,0.151844,-0.158051,1.000000,0.433410,-0.709160,0.107460
Real Estate,0.0510,0.114144,0.028545,0.148620,0.035470,0.354230,0.043955,0.432745,0.087186,0.174372,0.433410,1.000000,-0.361266,0.056465
Equity Hedges,0.0000,0.022508,0.058242,-0.179998,0.179479,-0.376090,0.201600,-0.476481,-0.069778,0.033113,-0.709160,-0.361266,1.000000,-0.008392


In [ ]:
#get data (policy weights, return, vol, corr, symbols) using inputs file
data_dict = util.get_data_dict(output_df)
data_dict.keys()

In [ ]:
#create plan object that has the relevant variables (cov, policy ret, var,fsv )
plan = pp.plan_params(data_dict['policy_weights'], data_dict['ret'], data_dict['vol'], data_dict['corr'], data_dict['symbols'])

In [ ]:
#TODO: import this data
bnds = pd.read_excel(ts.data_fp+'bounds.xlsx', sheet_name='bnds', index_col=0)
bnds =tuple(zip(bnds.Lower, bnds.Upper))
bnds

In [ ]:
cons = ({'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5},
        {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(plan)-2]) - .02})

In [ ]:
# Optimizing for maximum sharpe ratio
opt_sharpe = plan.optimize(plan.min_sharpe_ratio, bnds, cons)
opt_sharpe

In [ ]:
# Optimizing for minimum variance
opt_var = plan.optimize(plan.min_variance,bnds, cons)
opt_var
min_ret = plan.portfolio_stats(opt_var['x'])[0]
min_ret

In [ ]:
# Optimizing for max return
opt_ret = plan.optimize(plan.min_ret,bnds, cons)
opt_ret
max_ret = np.around(plan.portfolio_stats(opt_ret['x']),4)[0]
max_ret

In [ ]:
#Getting data for efficient frontier portfolios
target_rets = np.linspace(min_ret,max_ret,100)
# target_rets = np.linspace(-.01,0.04,100)
t_vols = []
t_weights = []

for tr in target_rets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: plan.portfolio_stats(x)[0] - tr},
               {'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5},
        {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(plan)-2]) - .02})
    
    opt_ef = plan.optimize(plan.min_volatility, bnds, ef_cons)
    
    t_vols.append(opt_ef['fun'])
    t_weights.append(opt_ef['x'])
    
target_vols = np.array(t_vols)
target_weights = np.array(t_weights)
optimized_weights = np.transpose(target_weights)
# opt_ef

In [ ]:
#Dataframe containing efficient frontier portfolios (return, vol, sharpe and weights)
df_ports = pd.DataFrame(np.column_stack([100*np.around(target_rets,6), 100*np.around(target_vols,6), np.around(target_rets/target_vols,6),100*np.around(target_weights,6)]),
                        columns=['Return', 'Volatility', 'Sharpe'] + plan.symbols)
df_ports.index.name = 'Portfolio'
df_ports['Portfolios'] = ['Portfolio']*100
df_ports['Portfolios'][df_ports[['Sharpe']].idxmax()[0]] = 'Max Sharpe Portfolio'
df_ports['Portfolios'][df_ports[['Volatility']].idxmin()[0]] = 'Min Var Portfolio'
df_ports

In [ ]:
#Asset Allocation Plot
combo_list = [('15+ STRIPS','forestgreen'),('Long Corporate','lightgreen'), ('Ultra 30-Year UST Futures','greenyellow'),
              ('Equity','deeppink'),('Liquid Alternatives','blue'),
('Private Equity','darkred'),('Credit','yellow'),('Real Estate','orange'),('Equity Hedges','blueviolet'),('Cash','khaki')]

fig = go.Figure()

for combo in combo_list:
    fig.add_trace(go.Scatter(
     x= df_ports['Return'], y = df_ports[combo[0]],
     name = combo[0],
     mode = 'lines',
     line=dict(width=0.5, color=combo[1]),
     stackgroup = 'one'))
fig.update_layout(
        title = {'text':"<b>Asset Allocation</b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
        title_font_family = "Calibri",
        titlefont = {"size":24},
        xaxis = dict(ticksuffix="%"),
        yaxis = dict(ticksuffix="%"),
#         legend_title="<b>Assets</b>",
        plot_bgcolor='White'
    )
fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>Returns</b>",
    title_font = {"size": 20},showline=True,linewidth=2,linecolor='black',mirror=False)
fig.update_yaxes(title_font_family = "Calibri",title_text = "<b>Weights</b>",range=(0,162),title_font = {"size": 20},
    showline=True,linewidth=2,linecolor='black',mirror=False)

fig.show()

In [ ]:
#Plotly version of the Efficient Frontier plot

fig = px.scatter(df_ports, x="Volatility", y="Return",color='Sharpe')
fig.update_layout(
    title={
            'text': "<b>Efficient Frontier Portfolio</b>",
            'y':0.9,
            'x':0.5,'xanchor': 'center',
            'yanchor': 'top'
            },
    title_font_family="Calibri",
    titlefont = {"size":20},
        xaxis = dict(tickfont = dict(size=14),ticksuffix="%"),
       yaxis = dict(ticksuffix="%"),
#     legend_title="Strategies",
    showlegend=True,
    plot_bgcolor='White'
                 )
fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>Expected Volatility</b>",title_font = {"size": 20},
                    showline=True,linewidth=2,linecolor='black',mirror=False)

fig.update_yaxes(title_font_family = "Calibri",title_text = "<b>Expected Return</b>",title_font = {"size": 20},
                 showline=True,linewidth=2,linecolor='black',mirror=False)
fig.show()

In [ ]:
#Export Efficient Frontier portfoio data to excel
cwd = os.getcwd()
report_path = cwd+'\\reports\\'
file_path = report_path +'ef_portfolios.xlsx'
writer = pd.ExcelWriter(file_path,engine='xlsxwriter')
pd.DataFrame(np.column_stack([target_rets,target_vols,target_rets/target_vols,target_weights]),
                        columns=['Return', 'Volatility', 'Sharpe'] + plan.symbols).to_excel(writer,sheet_name='AA')
writer.save()